In [ ]:
# Cell 1: Imports and display settings
from fenics import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


In [ ]:
# Cell 2: Geometry Parameters
radius = 1.83      # meters (12 ft diameter)
height = 4.27      # meters (14 ft height)
length = 1.83      # meters (symmetrical in z-direction)


In [ ]:
# Cell 3: Mesh
nx, ny, nz = 30, 60, 30
mesh = BoxMesh(Point(0, 0, 0), Point(radius, height, length), nx, ny, nz)


In [ ]:
# Cell 4: Function Spaces
V = FunctionSpace(mesh, "P", 1)
VV = VectorFunctionSpace(mesh, "P", 1)
W = VectorFunctionSpace(mesh, "P", 2)
Q = FunctionSpace(mesh, "P", 1)
VWQ = MixedFunctionSpace([W, Q])


In [ ]:
# Cell 5: Physical Constants
mu_0 = 4 * np.pi * 1e-7
T_c = 70
sigma_SB = 5.67e-8
q = 1.6e-19
k = 10.0
rho_cp = 1e6
h_cool = 500.0
emissivity = 0.9
nu = 1e-6
rho = 1e-5


In [ ]:
# Cell 6: Time-Stepping
dt = 0.05
t_end = 600.0
num_steps = int(t_end / dt)
time = 0.0


In [ ]:
# Cell 7: Initial Conditions
T_n = interpolate(Expression("300.0 - 50.0*x[1]", degree=1), V)
T = Function(V)
T.assign(T_n)

up0 = Expression(("0.0", "0.0", "0.0"), degree=2)
pp0 = Expression("0.0", degree=1)
up_init = interpolate(up0, W)
pp_init = interpolate(pp0, Q)
up_prev = Function(W)
pp_prev = Function(Q)
assign(up_prev, up_init)
assign(pp_prev, pp_init)


In [ ]:
# Cell 8: Source Fields
rho_plasma = interpolate(Expression("1e-5 + 1e-6*sin(5*x[0])*cos(3*x[2])", degree=2), V)

J_coil = interpolate(Expression(("1e6 * sin(8*pi*x[1]/height) * cos(2*pi*x[0]/radius)",
                                 "1e6 * cos(8*pi*x[1]/height) * sin(2*pi*x[0]/radius)",
                                 "0.0"),
                                height=height, radius=radius, degree=3), VV)

A_trial = TrialFunction(VV)
v_test = TestFunction(VV)
a_m = inner(curl(A_trial), curl(v_test)) * dx / mu_0


In [ ]:
# Cell 9: Variational Forms
upq = Function(VWQ)
(up, pp) = split(upq)
(v, q) = TestFunctions(VWQ)

J_plasma = q * rho * up
J_total = project(J_plasma + J_coil, VV)

L_m = inner(J_total, v_test) * dx
A_sol = Function(VV)
solve(a_m == L_m, A_sol)
B_field = project(curl(A_sol), VV)

f_L = cross(J_total, B_field)

a_NS = (1/dt)*inner(up, v)*dx + inner(dot(grad(up), up_prev), v)*dx + \
       nu*inner(grad(up), grad(v))*dx - inner(pp, div(v))*dx + inner(f_L, v)*dx + \
       inner(div(up), q)*dx
L_NS = (1/dt)*inner(up_prev, v)*dx


In [ ]:
# Cell 10: Temperature Variational Problem
T_trial = TrialFunction(V)
T_test = TestFunction(V)
f_L_mag = project(sqrt(dot(f_L, f_L)), V)

a_T = (T_trial * T_test + dt * k * dot(grad(T_trial), grad(T_test)) +
       dt * dot(up_sol, grad(T_trial)) * T_test) * dx

L_T = (T_n * T_test - dt * h_cool * (T_trial - Constant(30.0)) * T_test / radius +
       dt * f_L_mag * T_test) * dx


In [ ]:
# Cell 11: Output Files
vtkfile_T = File("output/Temperature.pvd")
vtkfile_B = File("output/MagneticField.pvd")


In [ ]:
# Cell 12: Time Loop
for n in range(num_steps):
    time += dt
    print(f"Time step {n+1}/{num_steps}, Time = {time:.2f} s")

    solve(lhs(a_NS) == rhs(L_NS), upq)
    up_sol, pp_sol = upq.split()
    assign(up_prev, up_sol)
    assign(pp_prev, pp_sol)

    J_plasma = q * rho * up_sol
    J_total = project(J_plasma + J_coil, VV)
    L_m = inner(J_total, v_test) * dx
    solve(a_m == L_m, A_sol)
    B_field = project(curl(A_sol), VV)
    f_L = project(cross(J_total, B_field), VV)
    f_L_mag = project(sqrt(dot(f_L, f_L)), V)

    A_T, b_T = assemble_system(a_T, L_T)
    solve(A_T, T.vector(), b_T)
    T_n.assign(T)

    if n % 50 == 0 or n == num_steps - 1:
        clear_output(wait=True)
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        c1 = plot(project(sqrt(dot(B_field, B_field)), V), title="|B| Field Strength")
        plt.colorbar(c1)
        plt.xlabel("x")
        plt.ylabel("y")

        plt.subplot(1, 2, 2)
        c2 = plot(T, title="Temperature")
        plt.colorbar(c2)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.suptitle(f"Time: {time:.2f} s")
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 13: Final Visualization
q_rad_expr = emissivity * sigma_SB * T**4
q_rad = project(q_rad_expr, V)
L_T += -dt * q_rad * T_test * dx

plt.figure(figsize=(10, 4))
c = plot(project(sqrt(dot(B_field, B_field)), V), title="Magnetic Field |B|", cmap="plasma")
plt.colorbar(c)
plt.xlabel("x")
plt.ylabel("y")
plt.show()
